In [2]:
import os
import cv2
from sklearn.neighbors import NearestNeighbors

def main():
    # Directory containing product images
    product_directory = r'C:\Users\DELL\Downloads\product_detection_from_packshots\product_detection_from_packshots\product_images'  
    
    # Directory containing shelf images
    shelf_directory = r'C:\Users\DELL\Downloads\product_detection_from_packshots\product_detection_from_packshots\shelf_images'  

    # Limit the number of product and shelf images for testing
    num_product_images = 5
    num_shelf_images = 5
    
    # Loop over a limited number of product images
    for product_filename in os.listdir(product_directory)[:num_product_images]:
        if product_filename.endswith('.jpg') or product_filename.endswith('.png'):
            # Construct the full path to the product image file
            product_image_path = os.path.join(product_directory, product_filename)
            
            # Loop over a limited number of shelf images
            for shelf_filename in os.listdir(shelf_directory)[:num_shelf_images]:
                if shelf_filename.endswith('.jpg') or shelf_filename.endswith('.png'):
                    # Construct the full path to the shelf image file
                    shelf_image_path = os.path.join(shelf_directory, shelf_filename)
                    
                    # Detect instances of the product in the shelf image
                    detect_instances(product_image_path, shelf_image_path)

def detect_instances(product_image_path, shelf_image_path):
    # Load product image and shelf image
    product_image = cv2.imread(product_image_path, cv2.IMREAD_GRAYSCALE)
    shelf_image = cv2.imread(shelf_image_path, cv2.IMREAD_GRAYSCALE)
    
    # Check if images are loaded correctly
    if product_image is None:
        print("Error: Failed to load product image from", product_image_path)
        return
    if shelf_image is None:
        print("Error: Failed to load shelf image from", shelf_image_path)
        return
    
    # Extract features from product image
    sift = cv2.SIFT_create()
    kp_product, des_product = sift.detectAndCompute(product_image, None)
    
    # Extract features from shelf image
    kp_shelf, des_shelf = sift.detectAndCompute(shelf_image, None)

    if des_product is None or des_shelf is None:
        print("No valid feature descriptors found in product or shelf image.")
        return
    
    # Find approximate nearest neighbors for each product feature in shelf images
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(des_shelf)
    distances, indices = nbrs.kneighbors(des_product)
    
    # Locate product in shelf image
    for distance, index in zip(distances, indices):
        if distance < 0.7:  # Threshold for matching features
            x_min, y_min, x_max, y_max = float('inf'), float('inf'), 0, 0
            for i in index:
                x, y = kp_shelf[i].pt
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x)
                y_max = max(y_max, y)
                
            # Write the detected instances to the output file
            with open('solution_1.txt', 'a') as f:
                f.write(f'{index},{i},{x_min},{y_min},{x_max},{y_max}\n')


In [12]:
import os
import cv2
import numpy as np

def load_images(folder_path, limit=None):
    images = []
    filenames = os.listdir(folder_path)
    if limit is not None:
        filenames = filenames[:limit]
    for filename in filenames:
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            images.append(image)
    return images

def find_products(shelf_images, product_images):
    results = []
    for shelf_idx, shelf_img in enumerate(shelf_images):
        for product_idx, product_img in enumerate(product_images):
            result = cv2.matchTemplate(shelf_img, product_img, cv2.TM_CCOEFF_NORMED)
            loc = np.where(result >= 0.9)
            for pt in zip(*loc[::-1]):
                x_min, y_min = pt
                x_max, y_max = x_min + product_img.shape[1], y_min + product_img.shape[0]
                results.append((shelf_idx, product_idx, x_min, y_min, x_max, y_max))
    return results

def write_solution(results, filename):
    with open(filename, 'w') as f:
        for result in results:
            shelf_idx, product_idx, x_min, y_min, x_max, y_max = result
            line = f"{shelf_idx},{product_idx},{x_min},{y_min},{x_max},{y_max}\n"
            f.write(line)

def main(num_product_images=10, num_shelf_images=10):
    # Directory containing product images
    product_directory = r'C:\Users\DELL\Downloads\product_detection_from_packshots\product_detection_from_packshots\product_images'
    
    # Directory containing shelf images
    shelf_directory = r'C:\Users\DELL\Downloads\product_detection_from_packshots\product_detection_from_packshots\shelf_images'
    
    # Load product images
    product_images = load_images(product_directory, limit=num_product_images)
    
    # Load shelf images
    shelf_images = load_images(shelf_directory, limit=num_shelf_images)
    
    # Find products in shelf images
    results = find_products(shelf_images, product_images)
    
    # Write solution to a file
    write_solution(results, 'solution_2.txt')


if __name__ == '__main__':
    main()


In [4]:
import cv2

def visualize_detections(shelf_image_path, detections, output_image_path):
    shelf_image = cv2.imread(shelf_image_path)
    
    # Draw bounding boxes on the shelf image
    for detection in detections:
        x_min, y_min, x_max, y_max = detection[2:6]  # Assuming the format of detection is (shelf_idx, product_idx, x_min, y_min, x_max, y_max)
        cv2.rectangle(shelf_image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    
    # Save the visualization
    cv2.imwrite(output_image_path, shelf_image)

def main():
    # Load shelf images
    shelf_images = [...]  # Load shelf images
    
    # Load detections from solution files
    detections_solution_1 = [...]  # Load detections from solution_1.txt
    detections_solution_2 = [...]  # Load detections from solution_2.txt
    
    # Visualize detections for solution 1
    for shelf_image_path, detections in zip(shelf_images, detections_solution_1):
        output_image_path = f"visualization_solution_1{shelf_image_path}"  # Define output image path
        visualize_detections(shelf_image_path, detections, output_image_path)
    
    # Visualize detections for solution 2
    for shelf_image_path, detections in zip(shelf_images, detections_solution_2):
        output_image_path = f"visualization_solution_2{shelf_image_path}"  # Define output image path
        visualize_detections(shelf_image_path, detections, output_image_path)

if __name__ == "__main__":
    main()


TypeError: Can't convert object to 'str' for 'filename'